In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
import warnings
warnings.filterwarnings('ignore')

In [88]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from backtesting import Backtest, Strategy
from strategies import PairTradeMeanReversion
import numpy as np
import pandas as pd
import numpy as np
import yfinance as yf
np.int = int

In [4]:
def create_pair(df_1: str, df_2: str) -> pd.DataFrame:
    df_1.index = pd.DatetimeIndex(df_1.index)
    df_2.index = pd.DatetimeIndex(df_2.index)
    pair_df = df_1.join(df_2, rsuffix='_y').dropna()
    pair_df.index = pair_df.index.tz_localize(None).astype('datetime64[ns]')
    return pair_df.reset_index()


In [59]:
import common.util as util

params = None

class PairTradeMeanReversion(Strategy):

    def init(self):
        self.ticks = 0
        self.trade_time = None
        self.ratio = self.I(lambda x: self.data['Ratio'], 0)

    def next(self):
        if self.data.index[-1] < 120:
            return
        
        is_outlier = 0
        hist = pd.Series(self.ratio[30:-2])
        if (self.ratio[-1] < hist.mean()-hist.std()):
            is_outlier = -1
        if (self.ratio[-1] > hist.mean()+hist.std()):
            is_outlier = 1

        t_delta = self.data.index[-1]-self.trade_time if self.trade_time else float('inf')
        if t_delta > 10:
            if (is_outlier < 0 and not params['is_denom']) or (is_outlier > 0 and params['is_denom']):
                self.trade_time = self.data.index[-1]
                self.buy()
            if (is_outlier > 0 and not params['is_denom']) or (is_outlier < 0 and params['is_denom']):
                self.trade_time = self.data.index[-1]
                self.sell()
        if abs(self.ratio[-1]-hist.mean()) < hist.std():
            self.position.close()

pairs = [
    ['AAPL', 'MSFT', ['2020-01-01', None]],
    # ['META', 'GOOG', ['2020-01-01', None]],
    # ['F', 'GM', ['2006-01-01', None]],
    # ['BA', 'LMT', ['2012-01-01', None]],
    # ['MRK', 'LLY', ['2007-01-01', '2018-01-01']],
]

for pair in pairs:
    print(pair)
    tkr_1 = pair[0]
    tkr_2 = pair[1]
    e_start = 5000

    df_1 = yf.Ticker(tkr_1).history(start=pair[2][0], end=pair[2][1])
    df_2 = yf.Ticker(tkr_2).history(start=pair[2][0], end=pair[2][1])

    params = {'is_denom': False}
    pair_1 = create_pair(df_1, df_2)
    ratio = util.norm_pairs(pair_1, 'Close', 'Close_y')
    pair_1['Ratio'] = ratio.copy()
    pair_1_bt = Backtest(pair_1, PairTradeMeanReversion,
                        cash=e_start, commission=.002,
                        exclusive_orders=True)
    output_1 = pair_1_bt.run()

    params = {'is_denom': True}
    pair_2 = create_pair(df_2, df_1)
    pair_2['Ratio'] = ratio.copy()
    pair_2_bt = Backtest(pair_2, PairTradeMeanReversion,
                        cash=e_start, commission=.002,
                        exclusive_orders=True)
    output_2 = pair_2_bt.run()

    pair_1_bt.plot(show_legend=False)
    pair_2_bt.plot(show_legend=False)

    print(f"trades 1: {output_1['# Trades']} trades 2: {output_2['# Trades']}")
    e_1 = output_1['Equity Final [$]']
    e_2 = output_2['Equity Final [$]']
    print(f"Final pair equity: {(e_1+e_2)/(2*e_start)*100}")
    print(f"Pair return %: {(e_1+e_2)/(2*e_start)*100}")
    print(f"wr 1 %: {output_1['Win Rate [%]']}, wr 2 %:{output_2['Win Rate [%]']}")
    bh_1 = df_1['Close'][-1]-df_1['Close'][0]
    bh_2 = df_2['Close'][-1]-df_2['Close'][0]
    print(f"Buy and hold return %: {(bh_1+bh_2)/(df_1['Close'][0]+df_2['Close'][0])*100}\n")

['AAPL', 'MSFT', ['2020-01-01', None]]


TypeError: 'Backtest' object is not subscriptable

In [91]:
import backtrader as bt

import common.util as util

class PairTradeMeanReversion(bt.Strategy):
    params = dict(ratio=pd.Series())

    def __init__(self):
        self.trade_time = None
        self.close_1 = self.datas[0].close
        self.close_2 = self.datas[1].close

    def next(self):
        i = len(self.close_1)-1
        if i < 60:
            return
        
        is_outlier = 0
        hist = pd.Series(self.p.ratio[30:i])
        if (self.p.ratio[i] < hist.mean()-hist.std()):
            is_outlier = -1
        if (self.p.ratio[i] > hist.mean()+hist.std()):
            is_outlier = 1

        t_delta = i-self.trade_time if self.trade_time else float('inf')
        if t_delta > 10:
            if is_outlier < 0:
                self.trade_time = i
                self.buy(data=self.datas[0])
                self.sell(data=self.datas[1])
            if is_outlier > 0:
                self.trade_time = i
                self.sell(data=self.datas[0])
                self.buy(data=self.datas[1])

pairs = [
    ['AAPL', 'MSFT', ['2020-01-01', None]],
    # ['META', 'GOOG', ['2020-01-01', None]],
    # ['F', 'GM', ['2006-01-01', None]],
    # ['BA', 'LMT', ['2012-01-01', None]],
    # ['MRK', 'LLY', ['2007-01-01', '2018-01-01']],
]

for pair in pairs:
    print(pair)
    tkr_1 = pair[0]
    tkr_2 = pair[1]
    e_start = 5000

    df_1 = yf.Ticker(tkr_1).history(start=pair[2][0], end=pair[2][1])
    df_2 = yf.Ticker(tkr_2).history(start=pair[2][0], end=pair[2][1])
    pair_df = create_pair(df_1, df_2)
    ratio = util.norm_pairs(pair_df, 'Close', 'Close_y')

    cerebro = bt.Cerebro()
    cerebro.addstrategy(PairTradeMeanReversion, ratio=ratio)
    cerebro.adddata(bt.feeds.PandasData(dataname=df_1))
    cerebro.adddata(bt.feeds.PandasData(dataname=df_2))
    cerebro.broker.setcash(100000.0)
    cerebro.run()
    cerebro.plot()

    # params = {'is_denom': False}
    # pair_1 = create_pair(df_1, df_2)
    # ratio = util.norm_pairs(pair_1, 'Close', 'Close_y')
    # pair_1['Ratio'] = ratio.copy()
    # pair_1_bt = Backtest(pair_1, PairTradeMeanReversion,
    #                     cash=e_start, commission=.002,
    #                     exclusive_orders=True)
    # output_1 = pair_1_bt.run()

    # params = {'is_denom': True}
    # pair_2 = create_pair(df_2, df_1)
    # pair_2['Ratio'] = ratio.copy()
    # pair_2_bt = Backtest(pair_2, PairTradeMeanReversion,
    #                     cash=e_start, commission=.002,
    #                     exclusive_orders=True)
    # output_2 = pair_2_bt.run()

    # pair_1_bt.plot(show_legend=False)
    # pair_2_bt.plot(show_legend=False)

    # print(f"trades 1: {output_1['# Trades']} trades 2: {output_2['# Trades']}")
    # e_1 = output_1['Equity Final [$]']
    # e_2 = output_2['Equity Final [$]']
    # print(f"Final pair equity: {(e_1+e_2)/(2*e_start)*100}")
    # print(f"Pair return %: {(e_1+e_2)/(2*e_start)*100}")
    # print(f"wr 1 %: {output_1['Win Rate [%]']}, wr 2 %:{output_2['Win Rate [%]']}")
    # bh_1 = df_1['Close'][-1]-df_1['Close'][0]
    # bh_2 = df_2['Close'][-1]-df_2['Close'][0]
    # print(f"Buy and hold return %: {(bh_1+bh_2)/(df_1['Close'][0]+df_2['Close'][0])*100}\n")

['AAPL', 'MSFT', ['2020-01-01', None]]


<IPython.core.display.Javascript object>